# Background
Wind power is one of the oldest and most recognizable types of renewable energy. Many readers may have memories of watching wind turbines through the window on flights or long rides in a car or train. And that is something to be noted; we don’t often see them. They are in areas we ignore. Wind turbines cannot be placed everywhere, and with heights of up to 400 feet, their vibrations adversely affect humans and animals. 

Despite wind power reliability, the approximately 54,000 turbines in the United States provided just 7.3% of the country’s energy in 2019. As shown in Figure 1, turbines are utilized in relatively few areas, especially when considering the United States’ greater than 1.75 million square miles of uninhabited land, as per the last census.

Of course, not all of this uninhabited land is ideal for installations. National parks, military bases, geographic entities, and more make wind power untenable in certain areas. Additionally, there needs to be enough wind to rotate the large rotors. 

### Research Questions


*   Where are the viable locations for the construction of wind turbines?
*   How much energy will be generated by the implementation of wind turbines in specific locations?
*   How does it affect humans in the future by replacing fossil fuels?

# Summary 
### Motivation
Our motivation for this project is to improve the environment and reduce the reliance on fossil fuels by locating viable areas where wind turbines can be constructed and calculate the potential energy production. What makes this important is because wind turbines are often disliked by humans and disturb local habitats. We need to find locations outside of large cities, away from protected areas, have a large amount of wind, and away from established wind turbines. Meeting these guidelines can be troublesome as identifying land with these traits can be somewhat problematic. 

### What This Project Is
This project is simply the first step in an extended process that (hopefully) ends in the proliferation of wind turbines in the United States. This project estimates optimal areas for wind turbine placements in the continental United States based on reasonable constraints. Additionally, taking a short analysis in some the benefits of a turbine(s) in the highest productivity location.

#### Identifying Turbine Placement Areas
We started with the continental United States and removed the areas turbines could not be placed in for identifying viable locations. Areas disqualified for this analysis in turbine placement are as follows:
*   Areas with existing turbines
*   Major cities (Population 10,000 and greater)
*   Designated protected areas
These disqualified areas are self-explanatory. It is not reasonable to place new turbines where existing turbines already are, or in or around large populations, or in national parks, military bases, protected wildlife areas, and so on. Our analysis determined potential turbine areas by systematically eliminating the disqualifying areas from contention, which reduced the entire area of the United States to only areas where turbine placement is practical.





# Data Sources
### [USGS United States Wind Turbines Dataset](https://eerscmap.usgs.gov/uswtdb/data/)
The USGS is one of the most comprehensive hosts of information about geographical data in the United States. Each row contains information like turbine location, size, energy output, rotor size, turbine manufacturer, turbine model, year online, and more.

### [ArcGIS United States Major Cities Dataset](https://hub.arcgis.com/datasets/esri::usa-major-cities?geometry=-107.388%2C31.327%2C-105.081%2C31.737)
An inherent problem with wind turbines is that they are enormous. Size plays a role in our analysis because they are a form of disturbance to landscape (physically and visually). Because of their size, turbines can cause nausea in nearby humans due to the generation of low-frequency vibrations. This dataset provides us information on the location (latitude/longitude), city population, and general demographics. 

### [NOAA United States Annual Wind Speed Dataset](https://www.climate.gov/maps-data/dataset/average-wind-speeds-map-viewer)
Knowing areas with high wind speeds is vital for efficient wind turbines. We will be pulling from two datasets with information on the average north-south and west-east wind speeds throughout the world. Knowing the values, we can calculate median wind directions and speed at a resolution of 2.5 degrees. 

### [USGS United States Protected Areas Dataset](https://www.usgs.gov/core-science-systems/science-analytics-and-synthesis/gap/science/pad-us-data-download?qt-science_center_objects=0#qt-science_center_objects)
As mentioned previously, there are limitations to the places where wind turbines can be located; we will be removing '2 - managed for biodiversity - disturbance events suppressed'. These locations in the United States are where there cannot be turbine constructions. These locations be National Parks, Wilderness areas, protected habitats, endangered species locations, etc.

### [United States Census Cartographic Boundary Dataset](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)
Cartographic Boundary Dataset is a collection of boundary files of the United States by the Census. For this analysis, we will be using the state’s boundaries at 20m resolution to focus on the entire United States without extensive geometry. This dataset has a total of 52 rows and nine columns. The extra two rows come from Puerto Rico and Guam.

### lat_lon_windspeed.csv and lat_lon_windspeed_REDUCED_RESOLUTION.csv
These two files were created during this project and are imperative to the results and visualizations. They contain the latitude, longitude, and annual wind speed at geographic intervals across the U.S. The first file contains points approximately every 700 feet, and the second file is at ¼ the first file’s resolution.


# Installs

In [ ]:
!pip install pyshp
!pip install netCDF4
!pip install pycrs
!pip install fiona
!pip install geopandas
!pip install shapely

     |████████████████████████████████| 225kB 5.7MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-cp36-none-any.whl size=37264 sha256=91acb4e07f9bf45c16945304068461fee3a634e99d21dd6d08e5a496bcad7e69
  Stored in directory: /root/.cache/pip/wheels/76/2b/d4/53e6b9a0fb0a9f9f29664cf82605af8bc81d5ab44d987896dd
Successfully built pyshp
     |████████████████████████████████| 4.7MB 5.7MB/s 
     |████████████████████████████████| 317kB 49.5MB/s 
  Created wheel for pycrs: filename=PyCRS-1.0.2-cp36-none-any.whl size=32684 sha256=fe2d21746ea7d22acf394019fa2bf8522cdd1bd9c65bc11b664f8b6b88806296
  Stored in directory: /root/.cache/pip/wheels/1a/c9/56/648f762bd41fdb03454be51b8a7069795925168833cb61f316
Successfully built pycrs
     |████████████████████████████████| 14.8MB 333kB/s 
     |████████████████████████████████| 972kB 4.2MB/s 
     |████████████████████████████████| 6.5MB 18.5MB/s 


# Imports

In [ ]:
import os
import requests
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import geopandas as gpd
import shapefile
from descartes import PolygonPatch
import netCDF4
import pycrs

# Files

In [ ]:
turbine_shp = 'source/Turbine_SHP/uswtdb_v3_3_20210114.shp'
cities_shp = 'source/USA_Major_Cities_SHP/0c5a2fa1-3463-4fc7-99e5-e206023a7e682020313-1-nmlntc.mln9c.shp'
boundaries_shp = 'source/US_State_Boundaries_SHP/cb_2018_us_state_20m.shp'
annual_wind_uwnd = 'source/Annual_Wind_speed/uwnd.sig995.2020.nc'
annual_wind_vwnd = 'source/Annual_Wind_speed/vwnd.sig995.2020.nc'
protected_shp = 'source/US_PADS_SHP/US_PADS_SHP_CON.shp'

assert os.path.exists(turbine_shp), "Turbine SHP file does not exist."
assert os.path.exists(cities_shp), "Cities over 10,000 SHP file does not exist."
assert os.path.exists(boundaries_shp), "US State Boundaries SHP file does not exist."
assert os.path.exists(annual_wind_uwnd), "Annual Wind Speed uwnd SHP file does not exist."
assert os.path.exists(annual_wind_vwnd), "Annual Wind Speed vwnd SHP file does not exist."
assert os.path.exists(protected_shp), "Protected Areas SHP file does not exist."

# Globals Variables

In [ ]:
PASS_STATES = set(['HI', 'AK', 'PR', 'GU'])
PADS_PASS_STATES = set(['Alaska', 'Puerto Rico', 'Unknown Federal', 'United States Virgin Islands', 'Hawaii', 'U.S. Minor Outlying Islands', 'Mariana Islands', 'American Samoa', 'Federated States of Micronesia', 'Palau'])
PADS_PASS_DES = set(['2 - managed for biodiversity - disturbance events suppressed'])
LAT_BTW_POINTS_MASTER = 0.0025
LON_BTW_POINTS_MASTER = 0.002975

# Turbine Data

The USWTDB is very well maintained and required little cleaning. At the time of our extraction of the data, there were no missing values. However, the database replaces unknown turbine radius (‘t_rd’) values with -9999.0. This value is essential because it determines the minimum distance at which other turbines can be placed. To alleviate this, we filtered out the rows that didn’t have a valid turbine radius as they made up a small portion of the dataset (< 10%). We also filtered out turbines, not in the continental U.S. We created a ‘buffer_radius’ field for the remaining records, which contains a turbine’s minimum distance to other turbines. The quantity is equal to 7 times the turbine radius, expressed in miles. Fields relevant to our analysis were ‘latitude,’ ‘longitude,’ ‘t_rd,’ and ‘buffer_radius.’ We collected, filtered, and extracted the shapefile data using the PyShp and stored it in a Pandas DataFrame for use in the project.

In [ ]:
def get_USWTDB_data():
  turbine_sf = shapefile.Reader(turbine_shp)
  items = list()
  for record, shape in zip(turbine_sf.records(), turbine_sf.shapes()):
      attributes = record.as_dict()
      if attributes['t_state'] not in PASS_STATES:
          items.append(attributes)
  turbine_df = pd.DataFrame(items)
  turbine_df.rename(columns={'xlong': 'longitude','ylat': 'latitude'}, inplace=True, errors='raise')
  turbine_df = turbine_df[turbine_df['t_rd'] != -9999.0]
  turbine_df['buffer_radius'] = (turbine_df['t_rd'] * 7 * 0.000621371) / 2 # now in miles
  return turbine_df

In [ ]:
get_USWTDB_data().head()

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,longitude,latitude,buffer_radius
14,3060912.0,19-028065,2014-WTE-4081-OE,-9999.0,-9999.0,IA,Story County,19169,30 MW Iowa DG Portfolio,2017.0,10.0,30.0,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0.0,-9999.0,3.0,3.0,2017-05-13,Digital Globe,-93.513710,42.019119,0.27185
15,3063321.0,19-028135,2014-WTE-4087-OE,-9999.0,-9999.0,IA,Hardin County,19083,30 MW Iowa DG Portfolio,2017.0,10.0,30.0,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0.0,-9999.0,3.0,3.0,2017-06-20,Digital Globe,-93.367798,42.497940,0.27185
16,3049500.0,19-028030,2014-WTE-4080-OE,-9999.0,-9999.0,IA,Story County,19169,30 MW Iowa DG Portfolio,2017.0,10.0,30.0,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0.0,-9999.0,3.0,3.0,2017-05-13,Digital Globe,-93.515892,42.016373,0.27185
17,3063269.0,19-028130,2016-WTE-5934-OE,-9999.0,-9999.0,IA,Story County,19169,30 MW Iowa DG Portfolio,2017.0,10.0,30.0,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0.0,-9999.0,3.0,3.0,2017-07-23,Digital Globe,-93.632835,41.882477,0.27185
18,3053390.0,19-028015,2015-WTE-6386-OE,-9999.0,-9999.0,IA,Boone County,19015,30 MW Iowa DG Portfolio,2017.0,10.0,30.0,Nordex,AW125/3000,3000.0,87.5,125.0,12271.85,150.0,0.0,-9999.0,3.0,3.0,2017-06-01,Digital Globe,-93.700424,41.977608,0.27185


# City Data
The Major Cities Dataset is similarly well-maintained. There were no missing values, and we used the PyShp and pandas libraries to extract and transform the data from the shapefile. Like above, we filtered the data to only include cities in the continental U.S. Important fields were ‘latitude,’ ‘longitude,’ and ‘POP_CLASS,’ which is a categorical designation for cities based on their populations. The data is stored in Pandas DataFrame for use in the project.

In [ ]:
def get_cities_data():
  city_sf = shapefile.Reader(cities_shp)
  items = list()
  for record, shape in zip(city_sf.records(), city_sf.shapes()):
      attributes = record.as_dict()
      if attributes['ST'] not in PASS_STATES:
          attributes['latitude'] = shape.points[0][1]
          attributes['longitude'] = shape.points[0][0]
          items.append(attributes)
  cities_df = pd.DataFrame(items)
  return cities_df

In [ ]:
get_cities_data().head()

,FID,NAME,CLASS,ST,STFIPS,PLACEFIPS,CAPITAL,POP_CLASS,POPULATION,POP2010,WHITE,BLACK,AMERI_ES,ASIAN,HAWN_PI,HISPANIC,OTHER,MULT_RACE,MALES,FEMALES,AGE_UNDER5,AGE_5_9,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_65_74,AGE_75_84,AGE_85_UP,MED_AGE,MED_AGE_M,MED_AGE_F,HOUSEHOLDS,AVE_HH_SZ,HSEHLD_1_M,HSEHLD_1_F,MARHH_CHD,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,latitude,longitude
0,1,Ammon,city,ID,16,1601990,,6,15181,13816,13002,73,67,113,9,884,307,245,6750,7066,1468,1503,1313,1058,734,2031,1767,1446,1136,665,486,209,29.6,28.0,30.8,4476,3.05,457,648,1618,1131,106,335,3352,3.61,4747,271,3205,1271,43.475792,-111.954103
1,2,Blackfoot,city,ID,16,1607840,,6,11946,11899,9893,40,418,125,18,2192,1077,328,5907,5992,1239,1099,890,817,818,1799,1235,1330,1143,721,579,229,30.8,29.7,32.1,4229,2.74,563,690,1091,1081,174,381,2958,3.31,4547,318,2788,1441,43.193937,-112.345567
2,3,Boise City,city,ID,16,1608830,State,8,225405,205671,182991,3043,1404,6501,457,14606,5139,6136,101690,103981,13155,12933,12750,13959,16966,32135,27048,29595,24177,12176,7087,3690,35.3,34.4,36.5,85704,2.36,16605,18104,16708,21233,2414,5919,50647,2.97,92700,6996,52345,33359,43.599015,-116.230110
3,4,Burley,city,ID,16,1611260,,6,10727,10345,7984,45,103,74,5,3460,1795,339,5136,5209,1073,959,790,768,699,1445,1136,1134,935,679,464,263,30.9,29.6,32.3,3644,2.76,498,634,950,861,139,358,2499,3.37,3885,241,2183,1461,42.536741,-113.793293
4,5,Caldwell,city,ID,16,1612250,,7,53942,46237,35856,300,539,406,41,16347,7449,1646,22821,23416,4962,4397,3803,3779,3687,7571,5559,4744,3624,2296,1222,593,28.1,27.3,28.9,14895,3.00,1795,2250,4407,3113,686,1755,10776,3.51,16323,1428,9699,5196,43.661626,-116.685619


# PAD Data
PADS differs from the turbines and cities datasets because it contains areas (like national parks and military bases) rather than point locations (like wind turbines and cities). As such, PADS required a slightly modified approach from that of the turbines and cities datasets, although, like the others, the data are well-maintained, required little to no cleaning, and we were only concerned with the geographic quantities (not the attributes for each area). We used PyShp to extract the attributes, geographic polygons, and bounding boxes (rectangular geographic area approximations) into separate DataFrames. We also filtered these data to include only the continental U.S. and not include biodiversity disturbance event areas, which are available for wind turbine construction.

The ‘bbox’ attribute is the most important in this dataset. We expanded the attribute by tabulating the centroid, east-west span, and north-south span of each bounding box. These quantities are necessary to rule out these areas from the areas of potential turbine placement.


In [ ]:
def get_PADS_data():
  pad_sf = shapefile.Reader(protected_shp)
  
  pad_items = list()
  items = list()
  bbox_list = list()
  states = []
  
  i = 0
  for record, shape in zip(pad_sf.records(), pad_sf.shapes()):
    attributes = record.as_dict()
    if attributes['d_GAP_Sts'] not in PADS_PASS_DES:
      if attributes['d_State_Nm'] not in PADS_PASS_STATES:
        poly = pad_sf.shape(i).__geo_interface__
        bbox = pad_sf.shape(i).bbox
        states.append(attributes['d_State_Nm'])
        items.append(poly)
        pad_items.append(attributes)
        bbox_list.append(bbox)
    i += 1

  poly_df = pd.DataFrame(items)
  pad_df = pd.DataFrame(pad_items)
  bbox_df = pd.DataFrame(data={'name': states, 'bbox': bbox_list})
  poly_df['name'] = states
  return pad_df, poly_df, bbox_df

In [ ]:
pad_df, poly_df, bbox_df = get_PADS_data()

In [ ]:
pad_df.head(2)

,Category,d_Category,Own_Type,d_Own_Type,Own_Name,d_Own_Name,Loc_Own,Mang_Type,d_Mang_Typ,Mang_Name,d_Mang_Nam,Loc_Mang,Des_Tp,d_Des_Tp,Loc_Ds,Unit_Nm,Loc_Nm,State_Nm,d_State_Nm,Agg_Src,GIS_Src,Src_Date,GIS_Acres,Source_PAI,WDPA_Cd,Access,d_Access,Access_Src,Access_Dt,GAP_Sts,d_GAP_Sts,GAPCdSrc,GAPCdDt,IUCN_Cat,d_IUCN_Cat,IUCNCtSrc,IUCNCtDt,Date_Est,Comments,SHAPE_Leng,SHAPE_Area
0,Other,Other,FED,Federal,USBR,Bureau of Reclamation,United States Bureau of Reclamation,STAT,State,SPR,State Park and Recreation,Idaho Department of Parks and Recreation,SP,State Park,State Park,Lake Cascade State Park,Lake Cascade State Park - Blue Heron Campground,ID,Idaho,USGS_PADUS1_4Designation_USBR_PADUS1_3,IDPR_Managed_Land.shp,2006/03/15,9,,0,OA,Open Access,IDFG_PAD-US v.1.3.gdb,2016,3,3 - managed for multiple uses - subject to ext...,GAP,2011,Other Conservation Area,Other Conservation Area,GAP - Default,2018,1996,,943.478776,35110.679531
1,Other,Other,FED,Federal,USBR,Bureau of Reclamation,United States Bureau of Reclamation,STAT,State,SPR,State Park and Recreation,Idaho Department of Parks and Recreation,SP,State Park,State Park,Lake Cascade State Park,Lake Cascade State Park - Boulder Creek Campgr...,ID,Idaho,USGS_PADUS1_4Designation_USBR_PADUS1_3,IDPR_Managed_Land.shp,2006/03/15,18,,0,OA,Open Access,IDFG_PAD-US v.1.3.gdb,2016,3,3 - managed for multiple uses - subject to ext...,GAP,2011,Other Conservation Area,Other Conservation Area,GAP - Default,2018,1996,,1425.473005,73292.653130


In [ ]:
poly_df.head()

,type,coordinates,name
0,Polygon,"[[(-116.05661287904096, 44.49500048372109), (-...",Idaho
1,Polygon,"[[(-116.09627998994497, 44.68758914187365), (-...",Idaho
2,Polygon,"[[(-116.11285006895555, 44.68253236619257), (-...",Idaho
3,Polygon,"[[(-116.05527337802039, 44.486643740358176), (...",Idaho
4,Polygon,"[[(-116.05674638184047, 44.526374244612384), (...",Idaho


In [ ]:
bbox_df.head()

,name,bbox
0,Idaho,"[-116.05761007614535, 44.49301757309738, -116...."
1,Idaho,"[-116.10057304403412, 44.68397039672821, -116...."
2,Idaho,"[-116.11541553423595, 44.681168688617745, -116..."
3,Idaho,"[-116.05619337402062, 44.48639687833194, -116...."
4,Idaho,"[-116.0596684958905, 44.52476859867801, -116.0..."


In [ ]:
"""
Returns:
  (longitude, latitude) of bbox centroid
"""
def calc_bbox_centroid(bbox):
  return np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])

"""
Returns:
  north-south span of bbox
"""
def north_south_span(bbox):
  return abs(bbox[1] - bbox[3])

"""
Returns:
  east-west span of bbox
"""
def east_west_span(bbox):
  return abs(bbox[0] - bbox[2])

In [ ]:
bbox_df['centroid'] = bbox_df.bbox.apply(calc_bbox_centroid)
bbox_df['ns_span'] = bbox_df.bbox.apply(north_south_span)
bbox_df['ew_span'] = bbox_df.bbox.apply(east_west_span)
bbox_df.head()

,name,bbox,centroid,ns_span,ew_span
0,Idaho,"[-116.05761007614535, 44.49301757309738, -116....","(-116.05574579513754, 44.49406382148888)",0.002092,0.003729
1,Idaho,"[-116.10057304403412, 44.68397039672821, -116....","(-116.09842651698955, 44.685783486279206)",0.003626,0.004293
2,Idaho,"[-116.11541553423595, 44.681168688617745, -116...","(-116.1138376304353, 44.683066523426575)",0.003796,0.003156
3,Idaho,"[-116.05619337402062, 44.48639687833194, -116....","(-116.05564615255929, 44.487266535949104)",0.001739,0.001094
4,Idaho,"[-116.0596684958905, 44.52476859867801, -116.0...","(-116.0572376830003, 44.52617750489104)",0.002818,0.004862


# Master Array
The master array is a two-dimensional NumPy array representing the United States. It is a matrix of latitude and longitude coordinates. There are 10,000 rows, with each row corresponding to a continually incremented latitude between 25.0 and 50.0 degrees. There are 20,000 columns, with each column corresponding to a continually incremented longitude between -126.0 and -66.5 degrees. The result is a matrix of 200,000,000 elements, where each element represents a geographic point in the U.S. This resolution results in approximately 700 feet between points-this are the minimum required distance between medium-large turbines.

The master array is created with every element initialized to 0. The analysis iterates over the tabulated disqualifying areas, the existing turbines, cities, and protected areas to determine the regions where turbine placement is feasible. Their minimum buffers-finds the corresponding coordinates in the master array and change those elements from 0 to 1. This results in the disqualifying area and its minimum buffer all set to 1 in the master array. Once all disqualifying areas have been labeled with 1s in the master array, the analysis tabulates the indices of all the remaining 0s. Where they are converted from indices to latitude-longitude coordinates, these coordinates represent the areas where wind turbines can be placed.

These valid locations from the master array are joined with the wind speed data. This join gives the average annual wind speed at each data point where wind turbine construction is feasible. The analysis sorts the dataset to find the yearly top-n wind speeds at valid locations to identify the optimal placements for future wind turbines.

This process uses exclusively numeric NumPy arrays. As a result, broadcasting and vectorization make this version of the analysis significantly faster, more accurate, and at a higher resolution than similar analyses using GeoPandas or other spatial data libraries.

The pseudocode to complete such an analysis is shown in Figure 2 below. While the actual analysis code addresses some particulars of the datasets, it is conceptually identical to the pseudocode.


In [ ]:
-120 -110 -100 -90 -80
      0 0 0 0 0 0     50
      0 0 0 0 0 0     40
      0 0 0 1 1 1     30
      0 0 0 1 1 1     20
      0 0 0 1 1 1     10
      0 0 0 0 0 0     0

master_latitudes[2] = 'the actual latitude for the 3rd row of master'

In [ ]:
master_latitudes = np.linspace(25.0, 50.0, num=10000)
master_longitudes = np.linspace(-126, -66.5, num=20000)

master_shape = (10000, 20000)
master = np.zeros(master_shape)

# Reduce Master with turbines (with buffer), cities (with buffer), and PADs

## Step 1. Base Map and Master Array
The green areas in the base map correspond to 0s in the master array. These areas are where a turbine could be placed. The entire map is green, and all array entries are 0 initially because the analysis hasn't disqualified any areas yet. It is important to remember that the map is just a cartographic representation of the array, and, similarly, the array is a numerical representation of the map.

## Step 2. Identify the disqualified areas
The red areas are a subset of the protected areas dataset, and the highlighted segments in the array are their approximate corresponding array locations. In the real analysis, all existing turbines, cities, protected areas, and buffers will be identified and removed from the base map/master array.

## Step 3. Subtract the disqualified areas
The cartographic representation will remove these areas from the green part of the map by turning them white (the yellow-green comes into play later-please ignore that aspect in the graphic below). The numerical representation will change the value of the corresponding elements from 0 to 1.




## Helper functions

In [ ]:
# latitude between points in master is 0.0025 degrees
def find_nearest_master_lat_index(latitude):
  upper_bound = 50.0
  lower_bound = 25.0
  num_steps = 10000
  step_size = abs(upper_bound - lower_bound) / num_steps
  nearest_master_index = int(np.rint((latitude - lower_bound) / step_size))
  return nearest_master_index

# longitude between points in master is 0.002975 degrees
def find_nearest_master_lon_index(longitude):
  upper_bound = -66.5
  lower_bound = -126
  num_steps = 20000
  step_size = abs(upper_bound - lower_bound) / num_steps
  nearest_master_index = int(np.rint((longitude - lower_bound) / step_size))
  return nearest_master_index

"""
Returns:
  miles converted to degrees latitude

Note:
  1 degree of latitude is approximately 69 miles
"""
def miles_to_latitude(miles):
  return miles / 69

"""
Returns:
  miles converted to degrees longitude
  
Note:
  1 degree of longitude is approximately 54.6 miles
"""
def miles_to_longitude(miles):
  return miles / 54.6

## Reducer Areas

In [ ]:
turbine_df = get_USWTDB_data()
cities_df = get_cities_data()

### Turbines

In [ ]:
# modifies master by turning all turbine indices into 1s
def turbine_indices():
  turbine_placement_cols = ['master_lat_index', 'master_lon_index', 'ew_buffer_points', 'ns_buffer_points']
  turbine_indices_w_buffers = turbine_df[turbine_placement_cols].to_numpy()
  for tiwb in turbine_indices_w_buffers:
    lat_i, lon_i = tiwb[0], tiwb[1]
    lat_dist, lon_dist = tiwb[2], tiwb[3]
    master[lat_i - lat_dist:lat_i + lat_dist + 1, lon_i - lon_dist:lon_i + lon_dist + 1] = 1
  return

In [ ]:
turbine_df['master_lat_index'] = turbine_df['latitude'].apply(find_nearest_master_lat_index)
turbine_df['master_lon_index'] = turbine_df['longitude'].apply(find_nearest_master_lon_index)

turbine_df['buffer_radius_lat'] = turbine_df.buffer_radius.apply(miles_to_latitude)
turbine_df['buffer_radius_lon'] = turbine_df.buffer_radius.apply(miles_to_longitude)

turbine_df['ew_buffer_points'] = (turbine_df.buffer_radius_lat / LAT_BTW_POINTS_MASTER).apply(np.rint).astype(int)
turbine_df['ns_buffer_points'] = (turbine_df.buffer_radius_lon / LON_BTW_POINTS_MASTER).apply(np.rint).astype(int)

In [ ]:
turbine_indices()
# MASTER IS NOW UPDATED WITH TURBINES

In [ ]:
#check that sum > 0
master.sum()

353271.0

### Cities

In [ ]:
# modifies master by turning all city indices into 1s
def cities_indices():
  city_placement_cols = ['master_lat_index', 'master_lon_index', 'ew_buffer_points', 'ns_buffer_points']
  city_indices_w_buffers = cities_df[city_placement_cols].to_numpy()
  for ciwb in city_indices_w_buffers:
    lat_i, lon_i = ciwb[0], ciwb[1]
    lat_dist, lon_dist = ciwb[2], ciwb[3]
    master[lat_i - lat_dist:lat_i + lat_dist + 1, lon_i - lon_dist:lon_i + lon_dist + 1] = 1
  return

In [ ]:
cities_df['master_lat_index'] = cities_df['latitude'].apply(find_nearest_master_lat_index)
cities_df['master_lon_index'] = cities_df['longitude'].apply(find_nearest_master_lon_index)

cities_df['buffer_radius_lat'] = cities_df.POP_CLASS.apply(miles_to_latitude)
cities_df['buffer_radius_lon'] = cities_df.POP_CLASS.apply(miles_to_longitude)

cities_df['ew_buffer_points'] = (cities_df.buffer_radius_lat / LAT_BTW_POINTS_MASTER).apply(np.rint).astype(int)
cities_df['ns_buffer_points'] = (cities_df.buffer_radius_lon / LON_BTW_POINTS_MASTER).apply(np.rint).astype(int)

In [ ]:
cities_indices()
# MASTER IS NOW UPDATED WITH CITIES

In [ ]:
#check that sum > 353271.0
master.sum()

11401101.0

### PADS

In [ ]:
# modifies master by turning all PADS indices into 1s
def pads_indices():
  pads_placement_cols = ['master_lat_index', 'master_lon_index', 'ew_buffer_points', 'ns_buffer_points']
  pads_indices_w_buffers = bbox_df[pads_placement_cols].to_numpy()
  for piwb in pads_indices_w_buffers:
    lat_i, lon_i = piwb[0], piwb[1]
    lat_dist, lon_dist = piwb[2], piwb[3]
    master[lat_i - lat_dist:lat_i + lat_dist + 1, lon_i - lon_dist:lon_i + lon_dist + 1] = 1
  return

In [ ]:
bbox_df['centroid_lat'] = bbox_df.centroid.apply(lambda x: x[1])
bbox_df['centroid_lon'] = bbox_df.centroid.apply(lambda x: x[0])

bbox_df['master_lat_index'] = bbox_df['centroid_lat'].apply(find_nearest_master_lat_index)
bbox_df['master_lon_index'] = bbox_df['centroid_lon'].apply(find_nearest_master_lon_index)

bbox_df['ew_buffer_points'] = (0.5 * bbox_df.ew_span / LAT_BTW_POINTS_MASTER).apply(np.rint).astype(int)
bbox_df['ns_buffer_points'] = (0.5 * bbox_df.ns_span / LON_BTW_POINTS_MASTER).apply(np.rint).astype(int)

In [ ]:
pads_indices()

In [ ]:
#check that sum > 11401101.0
master.sum()

55603910.0

## Indices where master == 0 (areas where turbines could be placed)

In [ ]:
lat_indices, lon_indices = np.where(master == 0)

# Wind Data
Like the preceding data sources, the NOAA data is exceptionally well-maintained and required little cleaning and manipulation. There are no null or missing values. Unlike the others, the wind speed data are formatted as netCDF. NetCDF files are standard for multidimensional and geographic data and are optimized for many-layered array-type data. The wind speed file we used contains NOAA’s wind speed data from 2020.

NOAA takes many wind speed readings across the globe each day. The file we used reports the daily averages for these readings from all 365 days in 2020. NOAA takes readings from latitudes -90 to 90 and longitudes 0 to 357.5, both in 2.5-degree increments, giving us 73 latitude points and 144 longitude points. Daily average wind speed readings are 73 x 144 = 10,512 coordinate pairings for each day in the timeframe. We used the year’s worth of data, which gave 10,512 * 365 = 3,836,880 wind speed readings. Additionally, wind speed is reported in U (East-West) and V (North-South) components. Therefore the data files contain 3,836,880 * 2 = 7,673,760 wind speed readings.

To extract the data from the files, we made heavy use of the netCDF4 library. We then used pandas and NumPy and created many new functions to arrange the data in meaningful ways and formats.

These functions culminate in mean_wind_speed_directory_table(), which extracts the data, combines all of the daily readings to determine the year’s U- and V-wind averages at each latitude-longitude interval, and arranges each of these into user-friendly DataFrames that are indexed by the geographic coordinates. The function eventually returns a dictionary with two keys: ‘U-wind’ and ‘V-wind,’ representing the corresponding wind component DataFrames.

We later combine the U- and V-components into a single magnitude by calculating their 2-norm. They are perpendicular vectors, and it is the magnitude of their combination which tells the story of true wind speed at a given location.



In [ ]:
f_uwnd = netCDF4.Dataset(annual_wind_uwnd)
f_vwnd = netCDF4.Dataset(annual_wind_vwnd)

In [ ]:
wind_lon = np.linspace(0, 357.5, num=144)
wind_lat = np.linspace(-90, 90, num=73)

### Wind Data Helper Functions

In [ ]:
"""
Parameters:
    file - path to valid NetCDF4 wind speed file; designed for
           NOAA U-wind and V-wind files; default 'drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'

Returns:
    mean_speed - a n_lat x n_lon numpy array representing the
                 average wind speed for each (n_lat, n_lon)
                 combination over the duration of the data
                 contained in the file
"""
def mean_wind_speed(file='drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'):
    f_wind = netCDF4.Dataset(file)
    n_lat = f_wind.dimensions['lat'].size
    n_lon = f_wind.dimensions['lon'].size
    n_days = f_wind.dimensions['time'].size
    key = list(f_wind.variables)[3]

    mean_speed = np.zeros((n_lat, n_lon))
    for day in f_wind.variables[key][:]:
        for i in range(n_lat):
            for j in range(n_lon):
                mean_speed[i][j] += day[i][j]
    mean_speed = mean_speed / n_days
    return mean_speed

"""
Parameters:
    file - path to valid NetCDF4 wind speed file; designed for
           NOAA U-wind and V-wind files; default 'drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'

Returns:
    numpy array of latitude values from file
"""
def lat_array(file='drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'):
    f_wind = netCDF4.Dataset(file)
    return f_wind.variables['lat'][:].data

"""
Parameters:
    file - path to valid NetCDF4 wind speed file; designed for
           NOAA U-wind and V-wind files; default 'drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'

Returns:
    numpy array of longitude values from file
"""
def lon_array(file='drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'):
    f_wind = netCDF4.Dataset(file)
    return f_wind.variables['lon'][:].data

"""
Parameters:
    file - path to valid NetCDF4 wind speed file; designed for
           NOAA U-wind and V-wind files; default 'drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'

Returns:
    table - a pandas dataframe pivot table where the row indices are
            the file's latitudes and the column indices are the file's
            longitudes; the values represent the mean wind speed for
            the corresponding latitude and longitude indices
"""
def mean_wind_speed_table(file='drive/MyDrive/data/Annual_Wind_speed/uwnd.sig995.2020.nc'):
    f_wind = netCDF4.Dataset(file)
    mws = mean_wind_speed(file)
    lat_lon_speed = list()

    lon, lat = f_wind.variables['lon'][:].data, f_wind.variables['lat'][:].data
    n_lat, n_lon = f_wind.dimensions['lat'].size, f_wind.dimensions['lon'].size

    for i in range(n_lat):
        for j in range(n_lon):
            d = dict()
            d['lat'] = lat[i]
            d['lon'] = lon[j]
            d['mean_speed'] = mws[i][j]
            lat_lon_speed.append(d)

    df = pd.DataFrame(lat_lon_speed)
    table = pd.pivot_table(df, index='lat', columns='lon')
    return table

"""
Parameters:
    dir_path - path to directory of netcdf4 wind speed files;
               default 'drive/MyDrive/data/Annual_Wind_speed/'

Returns:
    Python list containing two elements; the first element is
    a list of the U-wind component files in dir_path, and the
    second element is a list of the V-wind component files in
    dir_path
"""
def u_and_v_files(dir_path='drive/MyDrive/data/Annual_Wind_speed/'):
    files = os.listdir(dir_path)
    if not dir_path.endswith('/'):
        dir_path += '/'
    u_files = [f'{dir_path}{file}' for file in files if 'uwnd' in file]
    v_files = [f'{dir_path}{file}' for file in files if 'vwnd' in file]
    return u_files, v_files

"""
Parameters:
    dir_path - path to directory of netcdf4 wind speed files;
               default 'drive/MyDrive/data/Annual_Wind_speed/'

Returns:
    n_lat - number of latitude dimensions in files in dir_path
    n_lon - number of longitude dimensions in files in dir_path
"""
def lat_lon_dims_directory(dir_path='drive/MyDrive/data/Annual_Wind_speed/'):
    u_files, v_files = u_and_v_files(dir_path)
    file = u_files[0] # could take any file from the bunch
    f_wind = netCDF4.Dataset(file)
    n_lat, n_lon = f_wind.dimensions['lat'].size, f_wind.dimensions['lon'].size
    return n_lat, n_lon

"""
Parameters:
    dir_path - path to directory of netcdf4 wind speed files;
               default 'drive/MyDrive/data/Annual_Wind_speed/'

Returns:
    lat - np array of latitude dimensions in files in dir_path
    lon - np array of longitude dimensions in files in dir_path
"""
def lat_lon_vals_directory(dir_path='drive/MyDrive/data/Annual_Wind_speed/'):
    u_files, v_files = u_and_v_files(dir_path)
    file = u_files[0] # could take any file from the bunch
    return lat_array(file), lon_array(file)

"""
Parameters:
    dir_path - path to directory of netcdf4 wind speed files;
               default 'drive/MyDrive/data/Annual_Wind_speed/'

Returns:
    u_dir_mean - a n_lat x n_lon numpy array representing the
                 average U-wind speed for each (n_lat, n_lon)
                 combination over the duration of the data
                 contained in the directory 'dir_path'
    v_dir_mean - a n_lat x n_lon numpy array representing the
                 average V-wind speed for each (n_lat, n_lon)
                 combination over the duration of the data
                 contained in the directory 'dir_path'
"""
def mean_wind_speed_directory(dir_path='drive/MyDrive/data/Annual_Wind_speed/'):
    u_files, v_files = u_and_v_files(dir_path)
    u_means = [mean_wind_speed(u_file) for u_file in u_files]
    v_means = [mean_wind_speed(v_file) for v_file in v_files]
    u_dir_mean, v_dir_mean = np.mean(u_means, axis=0), np.mean(v_means, axis=0)
    return u_dir_mean, v_dir_mean

"""
Parameters:
    dir_path - path to directory of netcdf4 wind speed files;
               default 'drive/MyDrive/data/Annual_Wind_speed/'

Returns:
    mean_wind_speed_dict - Python dictionary consisting of two entries:
                           'U-wind' and 'V-wind';
                            Each value is a pandas dataframe pivot table
                            where row indices are latitudes and column
                            indices are longitudes; the values represent
                            the mean wind speed for the corresponding
                            latitude and longitude indices and key
                            direction
"""
def mean_wind_speed_directory_table(mws_directory=None, dir_path='drive/MyDrive/data/Annual_Wind_speed/'):
    mean_wind_speed_dict = dict()
    if mws_directory:
        u_dir_mean, v_dir_mean = mws_directory
    else:
        u_dir_mean, v_dir_mean = mean_wind_speed_directory(dir_path)
    n_lat, n_lon = lat_lon_dims_directory(dir_path)

    u_lat_lon_speed, v_lat_lon_speed = list(), list()
    lat, lon = lat_lon_vals_directory(dir_path)
    for i in range(n_lat):
        for j in range(n_lon):
            u_dict, v_dict = dict(), dict()
            u_dict['lat'], v_dict['lat'] = lat[i], lat[i]
            u_dict['lon'], v_dict['lon'] = lon[j], lon[j]
            u_dict['mean_speed'], v_dict['mean_speed'] = u_dir_mean[i][j], v_dir_mean[i][j]
            u_lat_lon_speed.append(u_dict)
            v_lat_lon_speed.append(v_dict)

    u_df = pd.DataFrame(u_lat_lon_speed)
    v_df = pd.DataFrame(v_lat_lon_speed)
    u_table = pd.pivot_table(u_df, index='lat', columns='lon')
    v_table = pd.pivot_table(v_df, index='lat', columns='lon')
    mean_wind_speed_dict['U-wind'] = u_table
    mean_wind_speed_dict['V-wind'] = v_table

    return mean_wind_speed_dict

## Wind Data (cont.)

In [ ]:
mws_dict = mean_wind_speed_directory_table()

In [ ]:
mws_dict['U-wind'].head()

mean_speed                      ...                              
lon        0.0       2.5       5.0    ...     352.5     355.0     357.5
lat                                   ...                              
-90.0  -0.588238 -0.460140 -0.331055  ... -0.965351 -0.840703 -0.715562
-87.5  -2.103591 -1.638450 -1.179788  ... -3.511830 -3.041760 -2.571830
-85.0  -3.013309 -2.558098 -2.147253  ... -4.646267 -4.061619 -3.515562
-82.5  -3.958238 -3.494576 -3.109788  ... -5.925422 -5.175281 -4.517393
-80.0  -3.795210 -3.911126 -4.136830  ... -4.890351 -4.244295 -3.885915

[5 rows x 144 columns]

In [ ]:
mws_dict['V-wind'].head()

mean_speed                      ...                              
lon        0.0       2.5       5.0    ...     352.5     355.0     357.5
lat                                   ...                              
-90.0  -2.921267 -2.943802 -2.960774  ... -2.817957 -2.858450 -2.893802
-87.5  -5.485985 -5.534365 -5.560069  ... -5.192957 -5.318168 -5.413872
-85.0  -6.456337 -6.478520 -6.501830  ... -6.331196 -6.392041 -6.430774
-82.5  -7.092605 -7.227957 -7.394295  ... -6.728661 -6.865140 -6.977041
-80.0  -5.109647 -5.079436 -5.193098  ... -6.039154 -5.626619 -5.296267

[5 rows x 144 columns]

In [ ]:
uw_array = mws_dict['U-wind'].to_numpy()
vw_array = mws_dict['V-wind'].to_numpy()

In [ ]:
# magnitude of wind speed at each coordinate
wind_magnitude = np.sqrt(uw_array**2 + vw_array**2)

In [ ]:
wind_magnitude

array([[2.97990318, 2.97954657, 2.97922443, ..., 2.97872171, 2.97951617,
        2.98095932],
       [5.87546795, 5.77180348, 5.68386058, ..., 6.26895124, 6.12659875,
        5.99369021],
       [7.12490829, 6.96527716, 6.84722474, ..., 7.85314196, 7.5733043 ,
        7.32898552],
       ...,
       [2.07246366, 2.04497007, 2.01448057, ..., 2.14788562, 2.12697954,
        2.10177781],
       [1.02470706, 1.05248221, 1.08039268, ..., 0.93408604, 0.96627716,
        0.99442566],
       [1.26783621, 1.26887061, 1.26900993, ..., 1.26925763, 1.26828608,
        1.26833946]])

# Match Wind Data with Reduced Master

## Step 4. Join wind data for valid areas
The analysis maps annual wind speed onto the valid turbine locations to identify the windiest valid areas. The cartographic representation encodes annual wind speed onto color with the green-yellow scale shown. The numerical representation in the analysis is a series of associative NumPy arrays, but shown below is a simplified, hypothetical DataFrame that demonstrates the data's connections.


## *** Saving and Loading Checkpoint ***

In [ ]:
# need to save master_latitudes, master_longitudes, lat_indices, lon_indices, wind_magnitude
i = 0
for arr in [master_latitudes, master_longitudes, lat_indices, lon_indices, wind_magnitude]:
  names = ['master_latitudes', 'master_longitudes', 'lat_indices', 'lon_indices', 'wind_magnitude']
  with open(f'drive/MyDrive/data/{names[i]}.npy', 'wb') as f:
    np.save(f, arr)
  i += 1

In [ ]:
# RESTART/CLEAR RUNTIME IN COLAB

In [ ]:
# open necessary saved files
with open(f'drive/MyDrive/data/master_latitudes.npy', 'rb') as f:
    master_latitudes = np.load(f)

with open(f'drive/MyDrive/data/master_longitudes.npy', 'rb') as f:
    master_longitudes = np.load(f)

with open(f'drive/MyDrive/data/lat_indices.npy', 'rb') as f:
    lat_indices = np.load(f)

with open(f'drive/MyDrive/data/lon_indices.npy', 'rb') as f:
    lon_indices = np.load(f)

with open(f'drive/MyDrive/data/wind_magnitude.npy', 'rb') as f:
    wind_magnitude = np.load(f)

## Run transformation

In [ ]:
latitudes_for_new_turbines = np.apply_along_axis(lambda x: master_latitudes[x], axis=0, arr=lat_indices)
longitudes_for_new_turbines = np.apply_along_axis(lambda x: master_longitudes[x], axis=0, arr=lon_indices)

## *** Saving and Loading Checkpoint ***

In [ ]:
# SAVE HERE IF RAM CONSTRAINTS PROBLEMATIC
# need to save latitudes_for_new_turbines, longitudes_for_new_turbines
i = 0
for arr in [latitudes_for_new_turbines, longitudes_for_new_turbines]:
  names = ['latitudes_for_new_turbines', 'longitudes_for_new_turbines']
  with open(f'drive/MyDrive/data/{names[i]}.npy', 'wb') as f:
    np.save(f, arr)
  i += 1

## *** Saving and Loading Checkpoint ***

In [ ]:
# open necessary saved files
with open(f'drive/MyDrive/data/latitudes_for_new_turbines.npy', 'rb') as f:
    latitudes_for_new_turbines = np.load(f)

with open(f'drive/MyDrive/data/longitudes_for_new_turbines.npy', 'rb') as f:
    longitudes_for_new_turbines = np.load(f)

with open(f'drive/MyDrive/data/wind_magnitude.npy', 'rb') as f:
    wind_magnitude = np.load(f)

## Create final dataframe of [lat, lon, wind speed]

In [ ]:
def find_nearest_wind_lat_index(latitude):
  upper_bound = 90.0
  lower_bound = -90.0
  num_steps = 73
  step_size = 2.5
  nearest_wind_index = int(np.rint((latitude - lower_bound) / step_size))
  return nearest_wind_index

def find_nearest_wind_lon_index(longitude):
  upper_bound = 357.5
  lower_bound = 0.0
  num_steps = 144
  step_size = abs(upper_bound - lower_bound) / num_steps
  nearest_wind_index = int(np.rint((longitude - lower_bound) / step_size))
  return nearest_wind_index

def wind_speed_from_latlon_array(arr):
  lat, lon = arr[0], arr[1]
  return wind_magnitude[find_nearest_wind_lat_index(lat), find_nearest_wind_lon_index(lon)]

In [ ]:
lat_lon_wind_speed_df = pd.DataFrame(data={'latitude': latitudes_for_new_turbines, 'longitude': longitudes_for_new_turbines})
lat_lon_wind_speed_df.head()

,latitude,longitude
0,25.0,-126.000000
1,25.0,-125.997025
2,25.0,-125.994050
3,25.0,-125.991075
4,25.0,-125.988099


In [ ]:
lat_lon_wind_speed_df['wind_magnitude'] = lat_lon_wind_speed_df.apply(wind_speed_from_latlon_array, axis=1, raw=True)

In [ ]:
lat_lon_wind_speed_df.head()

# Save lat_lon_wind_speed_df to CSV
## Result
The real analysis followed the pattern shown in the example, which reduced the continental U.S. to areas where turbine construction is possible and then mapped annual wind speed data onto those remaining areas. The analysis then identified the areas with the highest annual wind speeds, which are 
the most optimal areas for new turbine placement. 


In [ ]:
lat_lon_wind_speed_df.to_csv('drive/MyDrive/data/lat_lon_wind_speed.csv', index=False)

# Visualizations
Pandas and Matplotlib to create the final visualizations. Utilizing the Pandas sample function to reduce our scatter plots' resolution and limit coordinates to the continental United States. The benefit to this is that we can process just a fraction of our total data and get the same results when we calibrate for image size resolution.

In [ ]:
lat_lon = 'source/lat_lon_wind_speed/lat_lon_wind_speed_REDUCED_RESOLUTION.csv'

lat_lon_df = pd.read_csv(lat_lon)
boundaries_sf = gpd.read_file(boundaries_shp)

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

boundary_pass_states = set(['Puerto Rico', 'Alaska', 'Hawaii'])

# Load and create basemap.
def load_states(pass_states):
    states_sf = shapefile.Reader(boundaries_shp)
    fig = plt.figure() 
    count = 0
    items = list()
    states = []

    for record in states_sf.records():
        attributes = record.as_dict()
        if attributes['NAME'] not in pass_states:
            poly = states_sf.shape(count).__geo_interface__
            ax = fig.gca() 
            ax.add_patch(PolygonPatch(poly, fc='None', ec='#000000', alpha=1, zorder=2 ))
            ax.axis('scaled')
            states.append(attributes['NAME'])
            items.append(poly)
        count += 1
    
    states_df = pd.DataFrame(items)
    states_df['name'] = states
    return plt, states_df

# Filter points outside of the United States and remove undesired states.
def filter_points(sf, df, pass_states):
    sf = gpd.read_file(boundaries_shp)
    sf_cleaned = sf[~sf['NAME'].isin(pass_states)]
    poly = sf_cleaned['geometry'].geometry.unary_union
    poly = list(poly)[1]
    x, y = poly.exterior.xy
    poly_list = Polygon([(x[i], y[i]) for i in range(0, len(y))])

    df['bounds'] = df.apply(lambda x: Point(x['longitude'], x['latitude']).within(poly_list), axis=1)
    df = df.loc[df.bounds, :]
    return df.drop(columns=['bounds'])

In [ ]:
# Be VERY careful with this, I have reduced the sample size to provide better performance, 
# but to get the full expierence 10 million through 50 million are are good clear sample sizes.
# If you choose to change this number ber aware that it could be a while berfore it is done.
values = filter_points(boundaries_shp, lat_lon_df.sample(10000), boundary_pass_states)
values.head()

In [ ]:
# Create simple map
states_plt, states_df = load_states(boundary_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], s=0.2)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

## Potential Locations for Turbines
After processing over 100,000 rows of data and sectioning out areas outside of protected areas, established wind turbines within a specified range, we are returned 50,000,000 rows and three columns of data as potential wind turbine locations. When converted into an image, these rows equate to a resolution of 10,000 by 5,000. 

### How did we generate this plot?
Creating a unary union of the 48 states dissolves the state boundaries, giving us a vision of the 48 continental states. Converting this list of boundaries into the exterior removes any potential artifacts from any states failing to connect accurately. And by creating a tupled coordinate list and passing it into Shapely’s Polygon function. Iterating through our coordinates list allows us to create a Point with the Shapely library and check to determine if the latitude and longitude are within the bounding borders. These points are logged into a DataFrame. From this DataFrame, we can visualize the points by plotting a simple state outline and using a scatter plot. This function allowed us to create this plot with only points inside the United States boundaries. We have the ability to set the color based on our wind magnitude, and this gives us the visualization to the right.




In [ ]:
# Create gradient map
states_plt, states_df = load_states(boundary_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], c=values['wind_magnitude'], cmap='summer', s=0.2)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

## Productivity in Montana
### Highest Average Wind Speeds in the United States
Zooming into Montana, we can see that a large portion of the state consists of the brightest yellow, meaning this location has the highest average wind speeds in the United States at about 8.5 MPH on average. Interestingly, this block of high wind speeds is, for the most part, cut out of the analysis because of currently established wind turbines. Luckily for us, in the Northern end of these high wind values is just over 3.5 million acres of mostly rural land prime for the establishment of wind turbines. We are analyzing the Montana 30-Meter Residential-Scale Wind Resource Map from the Office of Energy Efficiency and Renewable Energy, coming directly out the Rocky mountains' valleys in high gusts of winds pushing through rivers and riverbeds and over the top remaining hills. For the lower elevation points, there are very insignificant amounts of wind. 


In [ ]:
# Plot Montana
focus_pass_states = set(['Puerto Rico', 'Alaska', 'Hawaii', 'Oregon', 'Washington', 'Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'West Virginia', 'Wisconsin'])

states_plt, states_df = load_states(focus_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], c=values['wind_magnitude'], cmap='summer', s=10)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

## Additional High Performing Locations
### Texas and Florida
An enormous and sparsely populated swath of central and south Texas-spanning from Lubbock to the Rio Grande Valley-is among the windiest regions with turbine potential in the United States. Although some wind turbines and settlements exist in this area (shown as the small white squares in the yellowish region), the vast majority of the land is undeveloped and perfect for new wind turbines. The diverse landscape, which combines canyons and mountains with deserts, plains, and forests, has plentiful wind resources to offer. Similarly, the southern tip of Florida has great wind power potential in the area spanning from Miami across to the Gulf Coast. Winds from the Gulf and the Carribean swirl across the Sunshine State and out into the Atlantic Ocean, endowing the swampy southern areas with great wind resources.


In [ ]:
# Plot Florida
focus2_pass_states = set(['Puerto Rico', 'Alaska', 'Arizona', 'Arkansas', 'Mississippi', 'Alabama', 'Georgia', 'South Carolina', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'])

states_plt, states_df = load_states(focus2_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], c=values['wind_magnitude'], cmap='summer', s=10)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

In [ ]:
# Plot Texas
focus4_pass_states = set(['Puerto Rico', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'California', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wyoming'])

states_plt, states_df = load_states(focus4_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], c=values['wind_magnitude'], cmap='summer', s=10)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

In [ ]:
# Plot California
focus3_pass_states = set(['Puerto Rico', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wyoming'])

states_plt, states_df = load_states(focus3_pass_states)
states_plt.scatter(values['longitude'], values['latitude'], c=values['wind_magnitude'], cmap='summer', s=10)
states_plt.rcParams['figure.figsize'] = [30, 20]
states_plt.show()

# Conclusion

### Summary
Wind is a reliable but underutilized renewable energy source with great potential in the United States. This project took the first steps towards realizing this potential by identifying the country’s most optimal areas for new turbines. We found these areas through a systematic geographic analysis of the country, which excluded areas with existing turbines, cities, and designated protected areas. The remaining regions are visualized with a gradient color indicating mean annual wind speeds. We can see the most optimal locations for turbine construction by the brighter yellow color; the most notable states are Montana, central and south Texas, and southern Florida. 

The data used in this analysis come from reputable sources in the science and geographic communities, such as ESRI, U.S. National Oceanic and Atmospheric Administration (NOAA), and the United States Geological Survey (USGS). The datasets were combined across their geographic features using a method James coined aforementioned “master array.”

### Additional Insights
Beyond the optimal areas our analysis identifies, we discovered that much of the United States’ wind power potential lies offshore. While our study only considered land turbines, these offshore locations will undoubtedly play an essential role in wind power growth.

Slightly removed from the analysis, we also discovered that GIS is superior to programming-based approaches for spatial data science projects like this one. Although we created some useful techniques to accomplish this analysis, geographic information systems could produce a more comprehensive investigation of this subject.

### What Worked and What Didn’t Work
The project took several ideological iterations to run smoothly. The original intent to predominantly use geospatial Python libraries proved untenable due to the size of our datasets and suboptimal library conventions. The project also had graphical struggles, as our large datasets and relatively average computing resources struggled to render the detailed visualizations. Beyond these issues, the project ran smoothly. We developed the “master array,” which turned out to be a more optimal workaround for the geospatial library troubles. Although the master array did not alleviate our infrastructure issues for rendering visualizations, it gave us much greater control in determining the resolution of our analysis and results. Additionally, we were fortunate to work with well-maintained datasets that required little to no cleaning and manipulation.

### Next Steps
Identifying the areas feasible for wind turbine construction is the first of many steps needed for mass adoption of wind power. There are lingering questions about infrastructure and engineering and many discussions about turbine machinery and energy transmission and storage. There are financial questions of cost and who should put up the money for such ventures. There are environmental concerns-will turbines upset our wildlife? There are political questions-who gets turbines first? There are social questions-do we want wind turbines near our towns or in our counties? Most notably, there are layered, integrated questions, which concern several of these subject areas on top of additional geographical context.

There is much left to do before our society can comfortably scale wind power to its potential. This project has taken the first step in this journey towards a wind-powered sustainable future, and the authors hope for continued progress to that end.



